In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc = {'figure.figsize':(15,8)})
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import random as rd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from copy import deepcopy
from tqdm import tqdm
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [12]:
total_df = pd.read_csv ('combined.csv')
total_df

,Area,Item,Year,Production_ton,Import_Qt_ton,Export_Qt_ton,Import_Vl_1000$,Export_Vl_1000$,Agri_1000ha
0,Argentina,"Meat, cattle",1961,2145064.0,3792.0,10441.0,5150.0,17709.0,137829.0000
1,Argentina,"Meat, cattle",1962,2378826.0,3792.0,10441.0,5150.0,17709.0,136434.0000
2,Argentina,"Meat, cattle",1963,2605287.0,3792.0,10441.0,5150.0,17709.0,134875.0000
3,Argentina,"Meat, cattle",1964,2019240.0,3792.0,10441.0,5150.0,17709.0,133297.0000
4,Argentina,"Meat, cattle",1965,1995096.0,3792.0,10441.0,5150.0,17709.0,131780.0000
...,...,...,...,...,...,...,...,...,...
1293,United States of America,"Milk, whole fresh cow",2015,94618978.0,2884.0,85134.0,3032.0,72602.0,404718.7000
1294,United States of America,"Milk, whole fresh cow",2016,96366267.0,3522.0,81332.0,3626.0,69697.0,405264.6000
1295,United States of America,"Milk, whole fresh cow",2017,97761519.0,7546.0,70854.0,6945.0,62575.0,405810.3538
1296,United States of America,"Milk, whole fresh cow",2018,98687302.0,6856.0,82761.0,7172.0,68482.0,405810.3538


In [13]:
ratio_df = pd.read_csv ('combined_ratio.csv')
ratio_df

,Area,Item,Year,Production_ton,Import_Qt_ton,Export_Qt_ton,Import_Vl_1000$,Export_Vl_1000$
0,Argentina,"Meat, cattle",1961,15.563227,0.027512,0.075753,0.037365,0.128485
1,Argentina,"Meat, cattle",1962,17.435727,0.027794,0.076528,0.037747,0.129799
2,Argentina,"Meat, cattle",1963,19.316308,0.028115,0.077412,0.038184,0.131299
3,Argentina,"Meat, cattle",1964,15.148428,0.028448,0.078329,0.038636,0.132854
4,Argentina,"Meat, cattle",1965,15.139596,0.028775,0.079231,0.039080,0.134383
...,...,...,...,...,...,...,...,...
1293,United States of America,"Milk, whole fresh cow",2015,233.789489,0.007126,0.210354,0.007492,0.179389
1294,United States of America,"Milk, whole fresh cow",2016,237.786046,0.008691,0.200689,0.008947,0.171979
1295,United States of America,"Milk, whole fresh cow",2017,240.904447,0.018595,0.174599,0.017114,0.154198
1296,United States of America,"Milk, whole fresh cow",2018,243.185767,0.016895,0.203940,0.017673,0.168754


# Machine Learning

## Data pre processing

In [14]:
#Create the machine learning dataframe and transform Area and Items to dummie variables
ml_df = total_df[["Area","Item"]]
ml_df = pd.get_dummies(ml_df,columns=["Area","Item"])
ml_df.reset_index(inplace=True,drop=True)
ml_df.tail(2)

,Area_Argentina,Area_Brazil,Area_Canada,Area_France,Area_Germany,Area_Ireland,Area_Italy,Area_Mexico,Area_Spain,Area_United Kingdom of Great Britain and Northern Ireland,Area_United States of America,"Item_Meat, cattle","Item_Milk, whole fresh cow"
1296,0,0,0,0,0,0,0,0,0,0,1,0,1
1297,0,0,0,0,0,0,0,0,0,0,1,0,1


In [15]:
#Scale this columns and join into ml_df
columns_to_scale = ["Import_Qt_ton","Export_Qt_ton","Agri_1000ha","Production_ton"]
scaler = MinMaxScaler()
ml_df = ml_df.join(pd.DataFrame(scaler.fit_transform(total_df[columns_to_scale]),columns=columns_to_scale))
ml_df.head()

,Area_Argentina,Area_Brazil,Area_Canada,Area_France,Area_Germany,Area_Ireland,Area_Italy,Area_Mexico,Area_Spain,Area_United Kingdom of Great Britain and Northern Ireland,Area_United States of America,"Item_Meat, cattle","Item_Milk, whole fresh cow",Import_Qt_ton,Export_Qt_ton,Agri_1000ha,Production_ton
0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.001365,0.004675,0.301341,0.020534
1,1,0,0,0,0,0,0,0,0,0,0,1,0,0.001365,0.004675,0.298194,0.022896
2,1,0,0,0,0,0,0,0,0,0,0,1,0,0.001365,0.004675,0.294676,0.025184
3,1,0,0,0,0,0,0,0,0,0,0,1,0,0.001365,0.004675,0.291116,0.019263
4,1,0,0,0,0,0,0,0,0,0,0,1,0,0.001365,0.004675,0.287694,0.019019


In [16]:
X_train, X_test,y_train,y_test = train_test_split(ml_df.iloc[:,:-1],ml_df.iloc[:,-1],test_size=0.2,random_state=42)

In [17]:
print("X Train size:",X_train.shape[0],
      "\ny Train size:",y_train.shape[0],
      "\nX Test size:",X_test.shape[0],
      "\ny Test size:",y_test.shape[0])

X Train size: 1038 
y Train size: 1038 
X Test size: 260 
y Test size: 260


## Random Forest Regressor

In [18]:
# define list of Parameters
params = {'n_estimators':[50,100,300,500,800,1000],
                  "min_samples_split":[2,5,10,20,40],
                  "max_depth":[None,5,10,20,40,50]
                   }

# Hyper parameter tuning via Grid Search Cross Validation 
grid_rf = GridSearchCV(estimator= RandomForestRegressor(random_state=42),
                          param_grid= params,
                          n_jobs=-1,
                          cv=5,
                          verbose=4,
                          refit= True
                     )

# Fit
grid_rf.fit(X_train,y_train)

# print best training model & score
print('Best training model:',grid_rf.best_estimator_)
print('Best model Parameters',grid_rf.best_params_)
print('Best training model score:', grid_rf.best_score_)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
Best training model: RandomForestRegressor(n_estimators=50, random_state=42)
Best model Parameters {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Best training model score: 0.9830591327585475


In [19]:
# Predict Production and unscale back to original values
y_pred = grid_rf.predict(X_test)

# Calculate Mean Absolute Error
MAE_rf = mean_absolute_error(y_test,y_pred)
print(MAE_rf)

0.0062393555068920355


## XGBoost Regressor

In [20]:
# define parameters space to loop over
params = {'n_estimators':[20,40,80,160,340,500],
             'max_depth':[3,6,9],
             'gamma':[0.01,0.1],
             'learning_rate':[0.001,0.01,0.1,1]
             }

# Hyper parameter tuning via Grid Search Cross Validation 
grid_xgb = GridSearchCV(estimator=XGBRegressor(random_state=42),
                     param_grid=params,
                     refit= True,
                     n_jobs=-1,
                     cv=5,
                     verbose=4
                     )

# fit grid to training scaled set
grid_xgb.fit(X_train,y_train)


# print best training model & R squared score
print('Best training model ',grid_xgb.best_estimator_)
print('Best model Parameters',grid_xgb.best_params_)
print('Best training model score, coefficient of determination R squared', grid_xgb.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/compat.py:36: F

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versio

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versio

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versio

...

[CV 4/5] END max_depth=None, min_samples_split=2, n_estimators=50;, score=0.990 total time=   0.1s
[CV 2/5] END max_depth=None, min_samples_split=2, n_estimators=300;, score=0.968 total time=   0.6s
[CV 5/5] END max_depth=None, min_samples_split=2, n_estimators=500;, score=0.965 total time=   1.0s
[CV 3/5] END max_depth=None, min_samples_split=2, n_estimators=1000;, score=0.992 total time=   1.9s
[CV 2/5] END max_depth=None, min_samples_split=5, n_estimators=500;, score=0.969 total time=   0.9s
[CV 5/5] END max_depth=None, min_samples_split=5, n_estimators=800;, score=0.964 total time=   1.4s
[CV 1/5] END max_depth=None, min_samples_split=10, n_estimators=100;, score=0.984 total time=   0.2s
[CV 4/5] END max_depth=None, min_samples_split=10, n_estimators=100;, score=0.987 total time=   0.2s
[CV 3/5] END max_depth=None, min_samples_split=10, n_estimators=300;, score=0.989 total time=   0.5s
[CV 4/5] END max_depth=None, min_samples_split=10, n_estimators=500;, score=0.987 total time=   0

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


 n_estimators=100;, score=0.949 total time=   0.2s
[CV 1/5] END max_depth=50, min_samples_split=40, n_estimators=300;, score=0.923 total time=   0.5s
[CV 2/5] END max_depth=50, min_samples_split=40, n_estimators=500;, score=0.869 total time=   0.8s
[CV 5/5] END max_depth=50, min_samples_split=40, n_estimators=800;, score=0.909 total time=   1.2s
[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=20;, score=-8.569 total time=   0.0s
[CV 4/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=40;, score=-6.777 total time=   0.0s
[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=80;, score=-7.498 total time=   0.0s
[CV 1/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=340;, score=-1.767 total time=   0.2s
[CV 4/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=500;, score=-2.141 total time=   0.3s
[CV 1/5] END gamma=0.01, learning_rate=0.001, max_depth=6, n_estimators=340;, score=-1.741 total time=   

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Best training model  XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0.01, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=340, n_jobs=1,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
Best model Parameters {'gamma': 0.01, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 340}
Best training model score, coefficient of determination R squared 0.9774029917647298


In [21]:
# Predict Production and unscale back to original values
y_pred = grid_xgb.predict(X_test)

# Calculate Mean Squared Error
MAE_rf = mean_absolute_error(y_test,y_pred)
print(MAE_rf)

0.012576005874525288
[CV 1/5] END max_depth=None, min_samples_split=2, n_estimators=50;, score=0.988 total time=   0.1s
[CV 3/5] END max_depth=None, min_samples_split=2, n_estimators=300;, score=0.992 total time=   0.6s
[CV 1/5] END max_depth=None, min_samples_split=2, n_estimators=800;, score=0.990 total time=   1.5s
[CV 4/5] END max_depth=None, min_samples_split=2, n_estimators=1000;, score=0.990 total time=   1.9s
[CV 2/5] END max_depth=None, min_samples_split=5, n_estimators=800;, score=0.969 total time=   1.3s
[CV 5/5] END max_depth=None, min_samples_split=5, n_estimators=1000;, score=0.964 total time=   1.6s
[CV 3/5] END max_depth=None, min_samples_split=10, n_estimators=800;, score=0.989 total time=   1.3s
[CV 2/5] END max_depth=None, min_samples_split=20, n_estimators=50;, score=0.956 total time=   0.1s
[CV 4/5] END max_depth=None, min_samples_split=20, n_estimators=50;, score=0.977 total time=   0.1s
[CV 1/5] END max_depth=None, min_samples_split=20, n_estimators=100;, score=0

[CV 3/5] END max_depth=None, min_samples_split=2, n_estimators=100;, score=0.991 total time=   0.2s
[CV 5/5] END max_depth=None, min_samples_split=2, n_estimators=300;, score=0.964 total time=   0.6s
[CV 3/5] END max_depth=None, min_samples_split=2, n_estimators=800;, score=0.992 total time=   1.5s
[CV 1/5] END max_depth=None, min_samples_split=5, n_estimators=50;, score=0.987 total time=   0.1s
[CV 2/5] END max_depth=None, min_samples_split=5, n_estimators=50;, score=0.981 total time=   0.1s
[CV 3/5] END max_depth=None, min_samples_split=5, n_estimators=50;, score=0.988 total time=   0.1s
[CV 4/5] END max_depth=None, min_samples_split=5, n_estimators=50;, score=0.990 total time=   0.1s
[CV 1/5] END max_depth=None, min_samples_split=5, n_estimators=100;, score=0.988 total time=   0.2s
[CV 4/5] END max_depth=None, min_samples_split=5, n_estimators=100;, score=0.990 total time=   0.2s
[CV 3/5] END max_depth=None, min_samples_split=5, n_estimators=300;, score=0.990 total time=   0.5s
[CV 

[CV 2/5] END max_depth=None, min_samples_split=2, n_estimators=100;, score=0.969 total time=   0.2s
[CV 4/5] END max_depth=None, min_samples_split=2, n_estimators=300;, score=0.991 total time=   0.6s
[CV 2/5] END max_depth=None, min_samples_split=2, n_estimators=800;, score=0.970 total time=   1.5s
[CV 5/5] END max_depth=None, min_samples_split=2, n_estimators=1000;, score=0.965 total time=   1.9s
[CV 3/5] END max_depth=None, min_samples_split=5, n_estimators=800;, score=0.990 total time=   1.4s
[CV 1/5] END max_depth=None, min_samples_split=10, n_estimators=50;, score=0.982 total time=   0.1s
[CV 2/5] END max_depth=None, min_samples_split=10, n_estimators=50;, score=0.966 total time=   0.1s
[CV 3/5] END max_depth=None, min_samples_split=10, n_estimators=50;, score=0.988 total time=   0.1s
[CV 4/5] END max_depth=None, min_samples_split=10, n_estimators=50;, score=0.987 total time=   0.1s
[CV 2/5] END max_depth=None, min_samples_split=10, n_estimators=100;, score=0.956 total time=   0.2

## Neural Network

In [ ]:
X_train.shape

In [ ]:
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model = Sequential()
model.add(Dense(32, input_dim=16, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(X_train, y_train, epochs=50, batch_size=10)

In [ ]:
# Predict Production and unscale back to original values
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
MAE_rf = mean_absolute_error(y_test,y_pred)
print(MAE_rf)